<a href="https://colab.research.google.com/github/Orkthi/metodos-numericos-para-equacoes-diferenciais/blob/main/difus%C3%A3o_totalmente_impl%C3%ADcita_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np

# Opções multiplot

#var = 'N'
#valores_da_var = [200,300,400,500,600,700,800,900,1000]

#var = 'alpha'
#valores_da_var = [1e-3, 3e-3, 5e-3, 7e-3, 9e-3]

#var = 'k'
#valores_da_var = [1e-3, 3e-3, 5e-3, 7e-3, 9e-3]

#var = 'C0'
#valores_da_var = [2, 6, 10, 14, 18]

var = 'Ce'
valores_da_var = [0, 2, 4, 6, 8, 10, 12, 14]

# Definição dos parâmetros de entrada
t0 = 0
tf = 50
dt = 0.01

# Constantes do problema
alpha = 1e-3
k = 5e-3

# Comprimento do domínio computacional
L = 3

# Concentração inicial
C0 = 2

# Condição de contorno no início
Ce = 10

# Número de nós
N = 200



def gauss_jacobi(A, b, epsilon=0.01, max_iter=1000, verbose=False): # epsilon é o erro admitido
    global Ce
    # Inicializando as variáveis
    n = len(A)
    x_old = np.zeros(n)  # Vetor inicial de chutes

    for i in range(n):
        x_old[i] = Ce/2     # Chute inicial = C0/2

    x_new = np.zeros(n)  # Vetor para armazenar as atualizações

    # Critério de parada: máximo de iterações ou erro menor que epsilon
    for iteration in range(max_iter):
        for i in range(n):
            # Calcula a nova estimativa para a variável
            sum_terms = sum(A[i][j] * x_old[j] for j in range(n) if j != i)
            x_new[i] = (b[i] - sum_terms) / A[i][i]

        # Verifica o critério de parada (erro)
        if np.linalg.norm(x_new - x_old) < epsilon:
            if verbose:
                print("Convergência alcançada!\n")
            return x_new

        # Atualiza o vetor para a próxima iteração
        x_old = np.copy(x_new)

    if verbose:
        print("Número máximo de iterações alcançado.\n")
    return x_new

def simulador():
    # delta x
    dx = L/(N-1)

    # Constantes
    s = (alpha*dt)/(dx**2)
    a = (1 + 2*s +k*dt)




    # Número de passos de tempo
    Ntimes = int( (tf-t0)/dt )

    # Definindo a matriz geral
    C = np.zeros(shape=(Ntimes, N)) # = C[n][i] = C[tempo][espaço]

    # Aplicando as condições de contorno e condições iniciais
    for i in range(N):
        # No tempo t=0, todos os nós recebem valor de 0 (INICIAL)
        C[0][i] = C0

    for i in range(Ntimes):
        # Em todo tempo, no espaço x = 0, C = Ce (CONTORNO)
        C[i][0] = Ce

    # Criando matriz de Constantes:
    A = np.zeros(shape=(N-1, N-1))

    # Preenchendo a matriz de constantes:
    # No interior da matriz
    for i in range(0, N-2):
        A[i][i] = a
        A[i+1][i] = -s
        A[i][i+1] = -s

    # Nas extremidades
    A[0][1] = -s
    A[N-2][N-3] = -2*s
    A[N-2][N-2] = a

    # Vetor de resultados B (A*X = B)
    B = np.zeros(N-1)



    # calculando os pontos
    # Iteração do tempo
    for j in range(1, Ntimes):
        # Iteração do espaço
        for i in range(1, N-1):
            # preenchendo o vetor resultados
            for i in range(N-1):
                B[i] = C[j-1][i+1]

            B[0] += s* Ce

        # Resolvendo
        gauss_jacobi(A, B, (5/100)) # Sem verbose e com erro admissível de 5%

    times= np.zeros(Ntimes)
    times[0] = t0
    for i in range(1, Ntimes):
        times[i] = times[i-1] + dt

    X = np.zeros(N)
    for i in range(N):
        X[i] = i*dx


    return X, C

# ####################### Processamento inicial ####################### #
def main():
    global C0, Ce, k, alpha, N
    if var == 'C0':
        for i in valores_da_var:
            C0 = i
            X, C = simulador()
            dx = L / (N - 1)
            s = (alpha * dt) / (dx ** 2)
            print(f"s = {s} para C0 = {C0}")

            plt.plot(X, C[-1])

    if var == 'Ce':
        for i in valores_da_var:
            Ce = i
            X, C = simulador()
            dx = L / (N - 1)
            s = (alpha * dt) / (dx ** 2)
            print(f"s = {s} para Ce = {Ce}")

            plt.plot(X, C[-1])

    if var == 'k':
        for i in valores_da_var:
            k = i
            X, C = simulador()
            dx = L / (N - 1)
            s = (alpha * dt) / (dx ** 2)
            print(f"s = {s} para k = {k}")

            plt.plot(X, C[-1])

    if var == 'alpha':
        for i in valores_da_var:
            alpha = i
            X, C = simulador()
            dx = L / (N - 1)
            s = (alpha * dt) / (dx ** 2)
            print(f"s = {s} para alpha = {alpha}")
            plt.plot(X, C[-1])

    if var == 'N':
        for i in valores_da_var:
            N = i
            X, C = simulador()
            dx = L / (N - 1)
            s = (alpha * dt) / (dx ** 2)
            print(f"s = {s} para N = {N}")
            plt.plot(X, C[-1])


    plt.xlim(0, L)
    plt.ylim(0, 10)
    plt.title("Concentração pelo comprimento variando no tempo")
    plt.xlabel("Comprimento")
    plt.ylabel("Concentração")

    # legendas
    legendas = []
    for i in valores_da_var:
        legendas.append(var + ' = ' + str(i))

    plt.legend(legendas)

    plt.show()


main()